In [2]:
import random
import numpy as np
import pandas as pd

In [10]:
count = 0
large_data = []
for loop in range(10):
    count += 1
    customers1 = np.random.normal(loc = 1000, scale = 250, size = 8)
    customers2 = np.random.normal(loc = 2000, scale = 500, size = 2)
    interested = []
    for item in customers1:
        interested.append(round(item / 50))
    for item in customers2:
        interested.append(round(item / 50))

    print('this is simulation:', count)
    inventory = [[0, 15, 15, 15]]
    prices = [199, 299, 399, 499, 599, 699, 799, 899]
    for week_num in range(len(interested)):
        one_inventory = []
        if week_num == 0:
            for price_num in range(len(prices)):
                one_status = []
                curA, curB, curC = (inventory[0][1], inventory[0][1], inventory[0][1])
                for i in range(interested[week_num]):
                    one_status.append(get_row_simp(i, one_status, prices[price_num], curA, curB, curC))
                one_ending = [str(price_num), one_status[-1][1], one_status[-1][2], one_status[-1][3]]
                rev = prices[price_num] * (45 - one_ending[1] - one_ending[2] - one_ending[3])
                one_ending.append(rev)
                one_inventory.append(one_ending)
            inventory.append(one_inventory)
        else:
            for price_num in range(len(prices)):
                for status in inventory[week_num]:
                    if int(status[0][-1]) < price_num:
                        continue
                    one_status = []
                    curA, curB, curC = (status[1], status[2], status[3])
                    if week_num <= 5:
                        for i in range(interested[week_num]):
                            one_status.append(get_row_simp(i, one_status, prices[price_num], curA, curB, curC))
                    elif week_num >5:
                        for i in range(interested[week_num]):
                            one_status.append(get_row_simp1(i, one_status, prices[price_num], curA, curB, curC))
                    one_ending = [status[0] + str(price_num), one_status[-1][1], one_status[-1][2], one_status[-1][3]]
                    rev = prices[price_num] * (curA - one_ending[1] + curB - one_ending[2] + curC - one_ending[3]) + status[4]
                    one_ending.append(rev)
                    one_inventory.append(one_ending)
            inventory.append(one_inventory)
            
    max = 0
    for item in inventory[10]:
        if item[4] > max:
            max = item[4]
            pattern = item[0]
            left = (item[1], item[2], item[3])
    large_data.append((max, pattern, left))

this is simulation: 1
this is simulation: 2
this is simulation: 3
this is simulation: 4
this is simulation: 5
this is simulation: 6
this is simulation: 7
this is simulation: 8
this is simulation: 9
this is simulation: 10


In [11]:
large_data

[(31056, '7766666333', (0, 1, 0)),
 (32955, '6666663300', (0, 0, 0)),
 (32755, '6666654330', (0, 0, 0)),
 (31055, '5555553000', (0, 0, 0)),
 (31855, '6655555330', (0, 0, 0)),
 (32755, '7655550000', (0, 0, 0)),
 (32455, '6666653330', (0, 0, 0)),
 (28155, '6555543333', (0, 0, 0)),
 (29855, '7666553333', (0, 0, 0)),
 (30855, '6666543330', (0, 0, 0))]

In [12]:
res = []
for item in large_data:
    one_res = []
    for ch in item[1]:
        n = int(ch)
        one_res.append(n)
    res.append(one_res)
res
pd1 = pd.DataFrame(res)
pd1

,0,1,2,3,4,5,6,7,8,9
0,7,7,6,6,6,6,6,3,3,3
1,6,6,6,6,6,6,3,3,0,0
2,6,6,6,6,6,5,4,3,3,0
3,5,5,5,5,5,5,3,0,0,0
4,6,6,5,5,5,5,5,3,3,0
5,7,6,5,5,5,5,0,0,0,0
6,6,6,6,6,6,5,3,3,3,0
7,6,5,5,5,5,4,3,3,3,3
8,7,6,6,6,5,5,3,3,3,3
9,6,6,6,6,5,4,3,3,3,0


In [13]:
pd1.mean(axis = 0)

0    6.2
1    5.9
2    5.6
3    5.6
4    5.4
5    5.0
6    3.3
7    2.4
8    2.1
9    0.9
dtype: float64

In [14]:
from scipy import stats
print(stats.mode(pd1))

ModeResult(mode=array([[6, 6, 6, 6, 5, 5, 3, 3, 3, 0]], dtype=int64), count=array([[6, 7, 6, 6, 6, 6, 6, 8, 7, 7]]))


In [4]:
def get_row_simp(rowid, data, price, curA, curB, curC):
    if rowid == 0:
        storageA = curA
        storageB = curB
        storageC = curC
    else:
        storageA = data[rowid - 1][1]
        storageB = data[rowid - 1][2]
        storageC = data[rowid - 1][3]

    rand_temp1 = np.random.random()
    if rand_temp1 <= 1 / 3.0:
        intend_purchase = 0
    elif rand_temp1 <= 2 / 3.0:
        intend_purchase = 1
    else:
        intend_purchase = 2

    rand_temp2 = np.random.random()
    if intend_purchase == 0 and storageA == 0:
        if rand_temp2 <= 0.6:
            intend_purchase = -1
        elif rand_temp2 > 0.6 and rand_temp2 <= 0.9:
            intend_purchase = 1
            if storageB == 0:
                intend_purchase = -1
        else:
            intend_purchase = 2
            if storageC == 0:
                intend_purchase = -1
    elif intend_purchase == 1 and storageB == 0:
        if rand_temp2 <= 0.1:
            intend_purchase = 0
            if storageA == 0:
                intend_purchase = -1
        elif rand_temp2 > 0.1 and rand_temp2 <= 0.7:
            intend_purchase = -1
        else:
            intend_purchase = 2
            if storageC == 0:
                intend_purchase = -1
    elif intend_purchase == 2 and storageC == 0:
        if rand_temp2 <= 0.1:
            intend_purchase = 1
            if storageB == 0:
                intend_purchase = -1
        else:
            intend_purchase = -1

    wtp = np.random.uniform(0, 1000)
    if wtp < price:
        intend_purchase = -1

    if intend_purchase == 0:
        ending_storageA = storageA - 1
        ending_storageB = storageB
        ending_storageC = storageC
    elif intend_purchase == 1:
        ending_storageB = storageB - 1
        ending_storageA = storageA
        ending_storageC = storageC
    elif intend_purchase == 2:
        ending_storageC = storageC - 1
        ending_storageA = storageA
        ending_storageB = storageB
    elif intend_purchase == -1:
        ending_storageA = storageA
        ending_storageB = storageB
        ending_storageC = storageC
    row = [rowid, ending_storageA, ending_storageB, ending_storageC]
    return row

In [3]:
def get_row_simp1(rowid, data, price, curA, curB, curC):
    if rowid == 0:
        storageA = curA
        storageB = curB
        storageC = curC
    else:
        storageA = data[rowid - 1][1]
        storageB = data[rowid - 1][2]
        storageC = data[rowid - 1][3]

    rand_temp1 = np.random.random()
    if rand_temp1 <= 1 / 3.0:
        intend_purchase = 0
    elif rand_temp1 <= 2 / 3.0:
        intend_purchase = 1
    else:
        intend_purchase = 2

    rand_temp2 = np.random.random()
    if intend_purchase == 0 and storageA == 0:
        if rand_temp2 <= 0.6:
            intend_purchase = -1
        elif rand_temp2 > 0.6 and rand_temp2 <= 0.9:
            intend_purchase = 1
            if storageB == 0:
                intend_purchase = -1
        else:
            intend_purchase = 2
            if storageC == 0:
                intend_purchase = -1
    elif intend_purchase == 1 and storageB == 0:
        if rand_temp2 <= 0.1:
            intend_purchase = 0
            if storageA == 0:
                intend_purchase = -1
        elif rand_temp2 > 0.1 and rand_temp2 <= 0.7:
            intend_purchase = -1
        else:
            intend_purchase = 2
            if storageC == 0:
                intend_purchase = -1
    elif intend_purchase == 2 and storageC == 0:
        if rand_temp2 <= 0.1:
            intend_purchase = 1
            if storageB == 0:
                intend_purchase = -1
        else:
            intend_purchase = -1

    wtp = np.random.uniform(0, 600)
    if wtp < price:
        intend_purchase = -1

    if intend_purchase == 0:
        ending_storageA = storageA - 1
        ending_storageB = storageB
        ending_storageC = storageC
    elif intend_purchase == 1:
        ending_storageB = storageB - 1
        ending_storageA = storageA
        ending_storageC = storageC
    elif intend_purchase == 2:
        ending_storageC = storageC - 1
        ending_storageA = storageA
        ending_storageB = storageB
    elif intend_purchase == -1:
        ending_storageA = storageA
        ending_storageB = storageB
        ending_storageC = storageC
    row = [rowid, ending_storageA, ending_storageB, ending_storageC]
    return row